In [ ]:
import nltk
import numpy as np
import pandas as pd
import keras 
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.corpus import treebank
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential,Model
from keras.layers import LSTM, GRU, Dense, Input, Dropout, Bidirectional, Masking, Embedding, SimpleRNN, TimeDistributed, RNN, SpatialDropout1D
from tensorflow.keras.utils import to_categorical
from nltk.tree import Tree
from nltk.chunk import tree2conlltags, conlltags2tree
import seaborn as sns
from gensim.models import Word2Vec, KeyedVectors
import time
from collections import defaultdict
import random
from tensorflow.keras import *

In [ ]:
# nltk.download('universal_tagset')
# nltk.download('tagsets')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')

## Part 1: Part of Speech Tagging

## الف

In [ ]:
print(nltk.help.upenn_tagset())

In [ ]:
class PTBPOSLoader:
    # *الف*
    def __init__(self, set_tagset = True, test_portion = 0.1, validation_portion = 0.1):
        if set_tagset:
            self.ptb = list(treebank.tagged_sents(tagset='universal'))
        else:
            self.ptb = list(treebank.tagged_sents())
            
        self.test_portion = test_portion
        self.validation_portion = validation_portion
        self.split_train_val_test_set()
            
    # *ب* 
    def split_train_val_test_set(self):
        self.train_set, self.test_set = train_test_split(self.ptb, test_size = self.test_portion, random_state = 100)
        self.train_set, self.val_set = train_test_split(self.train_set, test_size = self.validation_portion, random_state = 100)
        return self.train_set, self.val_set, self.test_set
        
    def extract_all_words_tag(self):
        self.train_words_tag = [word_tag for record in self.train_set for word_tag in record]
        self.val_words_tag = [word_tag for record in self.val_set for word_tag in record]
        self.test_words_tag = [word_tag for record in self.test_set for word_tag in record]
        return self.train_words_tag, self.val_words_tag, self.test_words_tag
        
    def set_vocab_and_tagset(self):
        vocab = set([word_tag[0] for word_tag in self.train_words_tag])  
        tagset = sorted(list(set([pair[1] for pair in self.train_words_tag])))
        return vocab, tagset
        
        
class POSTagger:
    def __init__(self, dataset_loader, smoothing_const = 0.001):
#         nltk.download('universal_tagset')
#         nltk.download('tagsets')
#         nltk.download('punkt')
#         nltk.download('averaged_perceptron_tagger')
#         nltk.download('maxent_ne_chunker')
#         nltk.download('words')
        
        self.train_set, self.val_set, self.test_set = dataset_loader.split_train_val_test_set()
        
        self.emission_file_name = 'emission_ptb.csv'
        self.smoothing_const = smoothing_const
        self.train_words_tag, self.val_words_tag, self.test_words_tag = dataset_loader.extract_all_words_tag()
        self.vocab, self.tagset = dataset_loader.set_vocab_and_tagset()
        self.t = len(self.tagset)
        self.v = len(self.vocab)
        
        self.test_run_base = [tup for sent in self.test_set for tup in sent]
        self.test_tagged_words = [tup[0] for sent in self.test_set for tup in sent]
        
    def get_transition(self, curr_tag, prev_tag):
        count_prev_tag = len([tag for tag in self.all_tags if tag == prev_tag])
        count_prev_tag_curr_tag = 0
        # count_prev_tag_curr_tag = len([tagsets[i] for i in range(len(tagsets)-1) if (tagsets[i + 1] == curr_tag and tagsets[i] == prev_tag)])
        for i in range(len(self.all_tags)-1):
            if self.all_tags[i + 1] == curr_tag and self.all_tags[i] == prev_tag:
                count_prev_tag_curr_tag += 1
        return count_prev_tag_curr_tag / count_prev_tag
    
    def set_transition_matrix(self, print_transition_matrix = False):
        self.all_tags = [pair[1] for pair in self.train_words_tag]
        self.tags_matrix = np.zeros((len(self.tagset), len(self.tagset)), dtype='float32')
        for i, t1 in enumerate(list(self.tagset)):
            for j, t2 in enumerate(list(self.tagset)): 
                self.tags_matrix[i, j] = self.get_transition(t2, t1)
        self.trans_df = pd.DataFrame(self.tags_matrix, columns = list(self.tagset), index=list(self.tagset))
        if print_transition_matrix:
            print(self.trans_df)
        
    # Calculating Emission Probabilities
    # The B emission probabilities, $P(wi|ti)$, represent the probability, given a tag (say Verb),
    # that it will be associated with a given word.
    def get_emission(self, word, tag):
        pair_with_tag_t = [pair for pair in self.train_words_tag if pair[1] == tag]    
        word_with_tag_t = [pair[0] for pair in pair_with_tag_t if pair[0] == word[0]]
        return len(word_with_tag_t) / len(pair_with_tag_t)
    
    def set_emission_matrix(self, read_from_file = True, print_emission_matrix = False):
        if read_from_file:
            self.read_emission_csv()
        else:
            self.emission_matrix = np.zeros((len(self.test_words_tag), len(self.tagset)), dtype='float32')
            for i, w in enumerate(list(self.test_words_tag)):
                if i % 300 == 0:
                    print(f"round: {i}")
                for j, t2 in enumerate(list(self.tagset)): 
                    self.emission_matrix[i, j] = self.get_emission(w, t2)
            self.emission_df = pd.DataFrame(self.emission_matrix, columns = list(self.tagset), index=list(self.test_words_tag))
        
        if print_emission_matrix:
            print(self.emission_df)
        
    def save_emission_csv(self):
        self.emission_df.to_csv(self.emission_file_name, index = list(self.test_words_tag))
        
    def read_emission_csv(self):
        self.emission_df = pd.read_csv(self.emission_file_name).set_index('Unnamed: 0')

    # *پ*
    #not considering OOV words
    def vanilla_viterbi(self, read_emission_file = True):
        viterbi = []
        for key, word in enumerate(self.test_tagged_words):
            max_prob = -1
            best_tag = None
            for tag in self.tagset:
                if key == 0:
                    transition_p = self.trans_df.loc['.', tag]              
                else:
                    last_state = viterbi[-1]
                    transition_p = self.trans_df.loc[last_state, tag]

                emission_p = self.emission_df.iat[key, self.tagset.index(tag)]
                
                prob = emission_p * transition_p
                if (prob >= max_prob):
                    max_prob = prob
                    best_tag = tag
                    
            viterbi.append(best_tag)
        return list(zip(self.test_tagged_words, viterbi))
    
    # *پ* & *ث*
    #considering OOV words
    def smoothed_viterbi(self):
        viterbi = []
        for key, word in enumerate(self.test_tagged_words):
            max_prob = -1
            best_tag = None 
            for tag in self.tagset:
                if !key:
                    transition_p = self.trans_df.loc['.', tag]
                else:
                    last_state = viterbi[-1]
                    transition_p = self.trans_df.loc[viterbi[-1], tag]

                if (word in self.vocab):
                    emission_p = self.emission_df.iat[key, self.tagset.index(tag)]
                else:
                    emission_p = self.smoothing_const

                prob = emission_p * transition_p
                if (prob >= max_prob):
                    max_prob = prob
                    best_tag = tag

            viterbi.append(best_tag)
        return list(zip(self.test_tagged_words, viterbi))
    
    def apply_viterbi(self, read_emission_matrix = True, smoothing = True):
        self.set_transition_matrix()
        self.set_emission_matrix(read_from_file = read_emission_matrix)
        if smoothing:
            tagged_res = self.smoothed_viterbi()
        else:
            tagged_res = self.vanilla_viterbi()
        return tagged_res
    
    # *ت*
    def evaluate(self, tagged_res):
        res = [] 
        wrong_preds = []
        for i, j in zip(tagged_res, self.test_run_base):
            if i == j:
                res.append(1)
            
        for _, pair in enumerate(zip(tagged_res, self.test_run_base)):
            if pair[0]!=pair[1]:
                wrong_preds.append(pair)
                
        accuracy = len(res)/len(tagged_res)
        return accuracy, wrong_preds
    

In [ ]:
ptb_loader = PTBPOSLoader()
pos_tagger = POSTagger(ptb_loader)
viterbi_tag_res = pos_tagger.apply_viterbi()
accuracy, wrong_preds = pos_tagger.evaluate(viterbi_tag_res)
print(f"accuracy = {accuracy}")
for i in range(10):
    print(f"word: {wrong_preds[i][0][0]} | predicted: {wrong_preds[i][0][1]} | expected: {wrong_preds[i][1][1]}")


## ح & ج

In [ ]:
class RecurrentPOSTagger:
    def __init__(self):
        def get_XY(dataset):
            X = []
            Y = []
            for sentence in dataset:
                X_sentence = []
                Y_sentence = []
                for entity in sentence:         
                    X_sentence.append(entity[0])
                    Y_sentence.append(entity[1])
                X.append(X_sentence)
                Y.append(Y_sentence)
            return X, Y
        
        def get_num_of_words_and_tags(X, Y):
            num_of_words = len(set([word.lower() for sentence in X for word in sentence]))
            num_of_tags = len(set([word.lower() for sentence in Y for word in sentence]))
            return num_of_words, num_of_tags
        
        def verbose(records, num_words, num_tags):
            print("Total number of tagged sentences: {}".format(len(records)))
            print("Vocabulary size: {}".format(num_words))
            print("Total number of tags: {}".format(num_tags))
            
        ptb_pos_loader = PTBPOSLoader()
        self.train_set, self.val_set, self.test_set = ptb_pos_loader.split_train_val_test_set()
        ptb_pos_loader.extract_all_words_tag()
        self.vocab, self.tagset = ptb_pos_loader.set_vocab_and_tagset()
        self.t = len(self.tagset)
        self.v = len(self.vocab)
        
        self.X, self.Y = get_XY(self.train_set)
        self.X_val, self.Y_val = get_XY(self.val_set)
        self.X_test, self.Y_test = get_XY(self.test_set)
        
        self.num_words, self.num_tags = get_num_of_words_and_tags(self.X, self.Y)
        self.num_words_val, self.num_tags_val = get_num_of_words_and_tags(self.X_val, self.Y_val)
        self.num_words_test, self.num_tags_test = get_num_of_words_and_tags(self.X_test, self.Y_test)
        
#         verbose(self.X, self.num_words, self.num_tags)
#         verbose(self.X, self.num_words_val, self.num_tags_val)
#         verbose(self.X, self.num_words_test, self.num_tags_test)
        
    def initialize_tokenizer(self, data):
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(data)   
        encoded = word_tokenizer.texts_to_sequences(data) 
        return encoded
    
    def apply_padding(self, tokenized_x, tokenized_y):
        pad_x = pad_sequences(tokenized_x, maxlen = self.max_length, padding = "pre", truncating = "post")
        pad_y = pad_sequences(tokenized_y, maxlen = self.max_length, padding = "pre", truncating = "post")
        return pad_x, pad_y
    
    def prepare_xy(self, padding_max_length = 100):
        self.max_length = padding_max_length
        tokenized_X = self.initialize_tokenizer(self.X)
        tokenized_X_val = self.initialize_tokenizer(self.X_val)
        tokenized_X_test = self.initialize_tokenizer(self.X_test)
        tokenized_Y = self.initialize_tokenizer(self.Y)
        tokenized_Y_val = self.initialize_tokenizer(self.Y_val)
        tokenized_Y_test = self.initialize_tokenizer(self.Y_test)
        
        self.X, self.Y = self.apply_padding(tokenized_X, tokenized_Y)
        self.X_val, self.Y_val = self.apply_padding(tokenized_X_val, tokenized_Y_val)
        self.X_test, self.Y_test = self.apply_padding(tokenized_X_test, tokenized_Y_test)
        self.Y, self.Y_val, self.Y_test = to_categorical(self.Y), to_categorical(self.Y_val), to_categorical(self.Y_test)
        self.num_classes = self.Y.shape[2]
        
    def make_model(self, layer_type = "RNN", num_of_units = 128, embedding_size = 300):
        self.rnn_model = Sequential()
        self.rnn_model.add(Embedding(input_dim = self.v, output_dim = embedding_size, input_length = self.max_length, trainable = False))
        if layer_type == "RNN":
            self.rnn_model.add(SimpleRNN(num_of_units, return_sequences = True, dropout = 0.1, recurrent_dropout = 0.1))
        elif layer_type == "LSTM":
            self.rnn_model.add(LSTM(num_of_units, return_sequences = True, dropout = 0.1, recurrent_dropout = 0.1))
        elif layer_type == "GRU":
            self.rnn_model.add(GRU(num_of_units, return_sequences = True, dropout = 0.1, recurrent_dropout = 0.1))
        self.rnn_model.add(TimeDistributed(Dense(self.num_classes, activation='softmax')))
        
        self.rnn_model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
        print(self.rnn_model.summary())
        
    def fit_model(self, test_data = False, batch_size = 32, epochs = 10):
        if test_data:
            rnn_training = self.rnn_model.fit(self.X, self.Y, batch_size=batch_size, epochs=epochs, validation_data=(self.X_test, self.Y_test))
        else:
            rnn_training = self.rnn_model.fit(self.X, self.Y, batch_size=batch_size, epochs=epochs, validation_data=(self.X_val, self.Y_val))
        return rnn_training
        

### Effect of Hyperparameters on Accuracy

In [ ]:
def plot(values, accuracies, topic):
    fig, ax = plt.subplots(nrows=1, ncols=len(values))
    fig.set_figheight(3)
    fig.set_figwidth(15)
    for i in range(len(values)):
        ax[i].plot(accuracies[i].history["accuracy"], label = "Train")
        ax[i].plot(accuracies[i].history["val_accuracy"], label ="Test")
        ax[i].set_title(f'{topic} Size: {values[i]}\nval_accuracy={accuracies[i].history["val_accuracy"][-1]:.5f}')
        ax[0].set_ylabel("Accuracy")

    plt.show()
    fig.savefig(f"{topic}.png")
    
def plot_parapeters(models):
    plt.style.use('seaborn')

    batches = [16, 32, 64]
    pad_max_lengths = [50, 100]
    embedding_sizes = [100, 200, 300]
    units = [32, 64, 128]


    for model in models:
        print("MODEL:", model)
        batch_accuracies = dict()
        max_len_accuracies = dict()
        embedding_size_accuracy = dict()

        for i, sz in enumerate(embedding_sizes):
            rnn_model = RecurrentPOSTagger()
            rnn_model.prepare_xy(padding_max_length = 100)
            rnn_model.make_model(layer_type = model, num_of_units = 128, embedding_size = sz)
            hist = rnn_model.fit_model(batch_size = 32, epochs = 15)
            embedding_size_accuracy[i] = hist

        plot(embedding_sizes, embedding_size_accuracy, f"{model}_Embedding")

        for i, sz in enumerate(pad_max_lengths):
            rnn_model = RecurrentPOSTagger()
            rnn_model.prepare_xy(padding_max_length = sz)
            rnn_model.make_model(layer_type = model, num_of_units = 128, embedding_size = 300)
            hist = rnn_model.fit_model(batch_size = 32, epochs = 15)
            max_len_accuracies[i] = hist

        plot(pad_max_lengths, max_len_accuracies, f"{model}_MaxLen")

        for i, sz in enumerate(batches):
            rnn_model = RecurrentPOSTagger()
            rnn_model.prepare_xy(padding_max_length = 100)
            rnn_model.make_model(layer_type = model, num_of_units = 128, embedding_size = 300)
            hist = rnn_model.fit_model(batch_size = sz, epochs = 15)
            batch_accuracies[i] = hist

        plot(batches, batch_accuracies, f"{model}_Batch")

models = ["LSTM", "GRU"]    
plot_parapeters(models)


### Studing Hyperparameters

In [ ]:
def plot(hist, topic):
    fig = plt.figure()
    plt.plot(hist.history["accuracy"], label = "Train")
    plt.plot(hist.history["val_accuracy"], label ="Test")
    plt.title("Accuracy of Train and validation Data")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()
    fig.savefig(f"{topic}.png")

units = [32, 64, 128]
batches = [16, 32, 64]
pad_max_lengths = [50, 100]
embedding_sizes = [100, 300]
models = ["RNN", "LSTM", "GRU"]

f = open("tuning_results.txt", "a")

for model in models:
    f.write(f"--------------> LAYER TYPE: {model} <--------------\n")
    for batch in batches:
        for max_length in pad_max_lengths:
            for embedding_size in embedding_sizes:
                rnn_model = RecurrentPOSTagger()
                rnn_model.prepare_xy(padding_max_length = max_length)
                rnn_model.make_model(layer_type = model, num_of_units = 64, embedding_size = embedding_size)
                hist = rnn_model.fit_model(batch_size = batch, epochs = 10)
                f.write(f'Layer: {model}, Batch size: {batch}, Padding Max length: {max_length}, Embedding size: {embedding_size}\n')
                f.write(f'val_accuracy={hist.history["val_accuracy"][-1]:.5f}\n')
                f.write('-'*30)
                f.write('\n')
                
f.close()

### Examining effect of 3 different unit values on RNN, LSTM and GRU : 32, 64, 128

In [ ]:
for layer in ["RNN", "LSTM", "GRU"]:
    for unit in [32, 64, 128]:
        rnn_model = RecurrentPOSTagger()
        rnn_model.prepare_xy(padding_max_length = 100)
        rnn_model.make_model(layer_type = layer, num_of_units = unit, embedding_size = 300)
        hist = rnn_model.fit_model(batch_size = batch, epochs = 15)
        print(f'accuracy: {hist.history["accuracy"][-1]:.5f}')
        print(f'val accuracy: {hist.history["val_accuracy"][-1]:.5f}')
        plot(hist, f"layer_{layer}_unit_{unit}")

## چ

## Testing Model on Test dataset

### RNN

In [ ]:
rnn_model_test_ziad = RecurrentPOSTagger()
rnn_model_test_ziad.prepare_xy(padding_max_length = 100)
rnn_model_test_ziad.make_model("RNN", num_of_units = 128, embedding_size = 300)
hist = rnn_model_test_ziad.fit_model(test_data=True, batch_size = 64, epochs = 12)
print(f'accuracy: {hist.history["accuracy"][-1]:.5f}')
plot(hist, f"test_RNN")

### LSTM

In [ ]:
rnn_model_test_ziad = RecurrentPOSTagger()
rnn_model_test_ziad.prepare_xy(padding_max_length = 100)
rnn_model_test_ziad.make_model("LSTM", num_of_units = 64, embedding_size = 300)
hist = rnn_model_test_ziad.fit_model(test_data=True, batch_size = 32, epochs = 12)
print(f'accuracy: {hist.history["accuracy"][-1]:.5f}')
plot(hist, f"test_RNN")

### GRU

In [ ]:
rnn_model_test_ziad = RecurrentPOSTagger()
rnn_model_test_ziad.prepare_xy(padding_max_length = 100)
rnn_model_test_ziad.make_model("GRU", num_of_units = 64, embedding_size = 300)
hist = rnn_model_test_ziad.fit_model(test_data=True, batch_size = 32, epochs = 12)
print(f'accuracy: {hist.history["accuracy"][-1]:.5f}')
plot(hist, f"test_RNN")

## Part 2: Named entity Recognition

In [ ]:
from sklearn.metrics import classification_report
class NER:
    # *الف*
    def __init__(self, smoothing_const = '0.001'):              
        self.emission_file_name = 'NER_emission_15.csv'
        self.smoothing_const = smoothing_const

        self.ptb_dataset = treebank.tagged_sents()  
        self.word_ne = []
        for record in self.ptb_dataset:
            tree = nltk.ne_chunk(record)
            iob_tags = tree2conlltags(tree)
            tags = [pair[2] for pair in iob_tags]
            self.word_ne = self.word_ne + [(i[0], i[2]) for i in iob_tags]
        
        self.train_set, self.test_set = train_test_split(self.word_ne, test_size = 0.15, random_state = 100)

        self.rec_ne_test_run_base = [sent for sent in self.test_set]
        self.rec_ne_test_tagged_words = [sent[0] for sent in self.test_set]
        
        self.words_tag = [record for record in self.train_set]      
        self.ne_vocab = set([word_tag[0] for word_tag in self.word_ne])

        self.all_tags = [record[1] for record in self.train_set]
        self.ne_tagsets = sorted(list(set(self.all_tags)))

        self.ne_t = len(self.ne_tagsets)
        self.ne_v = len(self.ne_vocab)
        
    def save_emission_csv(self):
        self.rec_ne_emission_df.to_csv(self.emission_file_name, index = list(self.test_set))
        
    def read_emission_csv(self):
        self.rec_ne_emission_df = pd.read_csv(self.emission_file_name).set_index('Unnamed: 0')
    # *ب*
    def get_transition(self, curr_tag, prev_tag):
        count_prev_tag = len([tag for tag in self.all_tags if tag == prev_tag])
        count_prev_tag_curr_tag = 0
        for i in range(len(self.all_tags)-1):
            if self.all_tags[i + 1] == curr_tag and self.all_tags[i] == prev_tag:
                count_prev_tag_curr_tag += 1
        return count_prev_tag_curr_tag / count_prev_tag
    
    def set_transition_matrix(self, print_transition_matrix = False):
        self.ne_tags_matrix = np.zeros((len(self.ne_tagsets), len(self.ne_tagsets)), dtype='float32')
        for i, t1 in enumerate(list(self.ne_tagsets)):
            for j, t2 in enumerate(list(self.ne_tagsets)): 
                self.ne_tags_matrix[i, j] = self.get_transition(t2, t1)
                if "I-" in t2:
                    if ("B-" in t1 and t1[2:] != t2[2:]) or ("I-" in t1 and t1[2:] != t2[2:]) or "O" in t1:
                        transition_p = 0

        self.ne_trans_df = pd.DataFrame(self.ne_tags_matrix, columns = list(self.ne_tagsets), index=list(self.ne_tagsets))
        if print_transition_matrix:
            print(self.ne_trans_df)
        
    def get_emission(self, word, tag):
        pair_with_tag_t = [pair for pair in self.train_set if pair[1] == tag]    
        word_with_tag_t = [pair[0] for pair in pair_with_tag_t if pair[0] == word[0]]
        return len(word_with_tag_t) / len(pair_with_tag_t)
    
    def set_emission_matrix(self, read_from_file = True, print_emission_matrix = False):
        if read_from_file:
            self.read_emission_csv()
        else:
            self.rec_ne_emission_matrix = np.zeros((len(self.test_set), len(self.ne_tagsets)), dtype='float32')
            for i, w in enumerate(list(self.test_set)):
                if i % 300 == 0:
                    print(f"round: {i}")
                for j, t2 in enumerate(list(self.ne_tagsets)):
                    self.rec_ne_emission_matrix[i, j] = self.ne_get_emission(w, t2)
            self.rec_ne_emission_df = pd.DataFrame(self.rec_ne_emission_matrix, columns = list(self.ne_tagsets), index=list(self.test_set))

        if print_emission_matrix:
            print(self.rec_ne_emission_df)
      
    # *ب*
    def Viterbi_NER(self):
        state = []
        for key, word in enumerate(self.rec_ne_test_tagged_words):
            max_prob = -1
            best_tag = None
            for tag in self.ne_tagsets:
                if key == 0:
                    transition_p = self.ne_trans_df.loc['O', tag]
                else:
                    transition_p = self.ne_trans_df.loc[state[-1], tag]

                if (word in self.ne_vocab):
                    emission_p = self.rec_ne_emission_df.iat[key, self.ne_tagsets.index(tag)]
                else:
                    emission_p = self.smoothing_const

                prob = emission_p * transition_p
                if (prob >= max_prob):
                    max_prob = prob
                    best_tag = tag

            state.append(best_tag)
        return list(zip(self.rec_ne_test_tagged_words, state))
    
    def apply_viterbi(self, read_emission_matrix = True):
        self.set_transition_matrix()
        self.set_emission_matrix(read_from_file = read_emission_matrix)
        tagged_seq_ne = self.Viterbi_NER()
        return tagged_seq_ne

    # *پ*
    def evaluate(self, tagged_res):
        res = [] 
        for i, j in zip(tagged_res, self.test_run_base):
            if i == j:
                res.append(1)
        accuracy = len(res)/len(tagged_res)
        report = classification_report([x[1] for x in self.rec_ne_test_run_base], [y[1] for y in tagged_res])
        return accuracy, report
    

In [ ]:
ner = NER()
viterbi_ner_res = ner.apply_viterbi()
accuracy, report = ner.evaluate(viterbi_ner_res)
print(f"accuracy = {accuracy}")
print(report)

### Transition Matrix

In [ ]:
ne_tags_matrix = np.zeros((len(ne_tagsets), len(ne_tagsets)), dtype='float32')
for i, t1 in enumerate(list(ne_tagsets)):
    for j, t2 in enumerate(list(ne_tagsets)): 
        ne_tags_matrix[i, j] = ne_get_transition(t2, t1, train_set)[0]/ne_get_transition(t2, t1, train_set)[1]
ne_trans_df = pd.DataFrame(ne_tags_matrix, columns = list(ne_tagsets), index=list(ne_tagsets))
ne_trans_df